In [1]:
using Pkg
Pkg.activate(".")

 Activating environment at `~/code/hypergraph_modularities_code/Project.toml`


In [2]:
using HypergraphModularity

┌ Info: Precompiling HypergraphModularity [0c934d27-dd44-49d7-950f-bd4be7819e54]
└ @ Base loading.jl:1278
┌ Warning: Package HypergraphModularity does not have DelimitedFiles in its dependencies:
│ - If you have HypergraphModularity checked out for development and have
│   added DelimitedFiles as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with HypergraphModularity
└ Loading DelimitedFiles into HypergraphModularity from project dependency, future warnings for HypergraphModularity are suppressed.
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



In [3]:
mink, maxk = 2, 25
for dataset in ["contact-primary-school-classes", 
                "contact-high-school-classes", 
                "walmart-trips", 
                "HR-congress-bills",
                "SN-congress-bills",
                "TrivagoClickout"
                ]
    H, labels = read_hypergraph_data(dataset, maxk, mink)
    hedge_sizes = [sz_edge_key for (sz_edge_key, sz_edge_val) in H.E if length(sz_edge_val) > 0]
    min_size = minimum(hedge_sizes)
    max_size = maximum(hedge_sizes)
    num_nodes = sum(H.D .> 0)
    num_hedges = sum([length(sz_edge_val) for (sz_edge_key, sz_edge_val) in H.E])
    num_labels = length(unique(labels))
    println("$(dataset) & $(num_nodes) & $(num_hedges) & $(min_size)--$(max_size) & $(num_labels) \\\\")
end

contact-primary-school-classes & 242 & 12704 & 2--5 & 11 \\
contact-high-school-classes & 327 & 7818 & 2--5 & 9 \\
walmart-trips & 88860 & 65979 & 2--25 & 11 \\
HR-congress-bills & 1494 & 43047 & 2--25 & 2 \\
SN-congress-bills & 293 & 20006 & 2--25 & 2 \\
TrivagoClickout & 171495 & 220758 & 3--25 & 160 \\
